In [ ]:
# !mkdir /home/aistudio/external-libraries

In [ ]:
# !conda install cudatoolkit==9.2 catboost==0.18 tensorflow-gpu==1.14 keras -p /home/aistudio/external-libraries

In [ ]:
!pip install gensim lightgbm pandas==0.24.2

Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple/


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import zipfile
import gc
%matplotlib inline

In [4]:
pd.read_csv('submission.csv').astype(int).to_csv("./submission.csv", header = True, index = False, encoding='utf-8')

In [ ]:
!cat submission.csv

In [ ]:
# zipfile.ZipFile('data/data36883/腾讯.zip').extractall('./work')

In [ ]:
# zipfile.ZipFile('work/train_preliminary.zip').extractall('./work')
# zipfile.ZipFile('work/test.zip').extractall('./work')

### Load data

In [ ]:
# import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics

In [ ]:
import sys
sys.path.append('/home/aistudio/external-libraries/lib/python3.7/site-packages')

In [ ]:
from keras.layers import *
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.optimizers import Adam
from keras.models import Model, Sequential
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.layers import concatenate
from keras.callbacks import *

maxlen = 128
# max_features = 100000
# embed_size = 300

def create_embedding(texts, emb_save_file):
    embedding_wv = KeyedVectors.load_word2vec_format(
            emb_save_file, binary=False)
    embedding_matrix = np.zeros((len(texts), embed_size))
    for ix, word in enumerate(texts):
        try:
            embedding_vector = embedding_wv[word]
            embedding_matrix[i] = embedding_vector
        except:
            continue
            
    return embedding_matrix


def model_embedding(out_size, embedding_matrix):
    max_features, embed_size = embedding_matrix.shape
    model = Sequential()
    model.add(Embedding(max_features, embed_size, 
                embeddings_initializer = Constant(embedding_matrix),
                input_length = maxlen,
                trainable=False))
    
    model.add(Flatten())
    model.add(Dense(128))
    if out_size == 2:
        model.add(Dense(out_size, activation='sigmoid'))
        
    else:
        model.add(Dense(out_size, activation='softmax'))
    # model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
    # model.summary()
    return model

def model_lstm_atten(out_size, embedding_matrix):
    '''
    maxlen # max number of words in a question to use
    '''
    max_features, embed_size = embedding_matrix.shape
    
    inp = Input(shape = (maxlen,))
    
    x = Embedding(max_features, embed_size, 
                embeddings_initializer = Constant(embedding_matrix),
                input_length = maxlen,
                trainable = False)(inp)
    x = SpatialDropout1D(0.1)(x)
    x = Bidirectional(CuDNNLSTM(128, return_sequences = True))(x)
    y = Bidirectional(CuDNNGRU(64, return_sequences = True))(x)
    z = Conv1D(64, kernel_size = 1, kernel_initializer = initializers.he_uniform(seed=2020), activation = "tanh")(y)
    
    atten_1 = Attention(maxlen)(x) # skip connect
    atten_2 = Attention(maxlen)(y)
    avg_pool = GlobalAveragePooling1D()(y)
    max_pool = GlobalMaxPooling1D()(y)
    max_pool1 = GlobalMaxPooling1D()(z)
    
    convs = []
    filter_sizes = [2, 3, 5, 8]
    for fsz in filter_sizes:
        l_conv = Conv1D(filters=maxlen, kernel_size=fsz, activation='relu')(y)
        l_pool = MaxPooling1D(maxlen - fsz + 1)(l_conv)
        l_pool = Flatten()(l_pool)
        convs.append(l_pool)
    text_cnn = concatenate(convs, axis=1)
    
    
    conc = concatenate([atten_1, atten_2, avg_pool, max_pool, max_pool1, text_cnn])
    conc = Dense(256)(conc)
    conc = BatchNormalization()(conc)
    conc = PReLU()(conc)
    conc = Dropout(0.2)(conc)
    
    conc = Dense(128)(conc)
    conc = BatchNormalization()(conc)
    if out_size == 2:
        outp = Dense(out_size, activation="sigmoid")(conc) 
    else:
        outp = Dense(out_size, activation="softmax")(conc)    

    model = Model(inputs = inp, outputs = outp)
    
    return model
    
cb = [EarlyStopping(monitor='val_loss', patience = 3, verbose = 1),]
               
def run_oof_nn(X, y, X_test, kfold, label_size):
    models = []
    pred = np.zeros((X_test.shape[0], label_size))
    oof = np.zeros((X.shape[0], label_size))
    for index, (trn_idx, val_idx) in enumerate(kfold.split(X=X, y=y)):
        K.clear_session()
        print('-' * 88)
        x_trn, y_trn = X[trn_idx], to_categorical(y.iloc[trn_idx], num_classes = label_size, dtype='int8')
        x_val, y_val = X[val_idx], to_categorical(y.iloc[val_idx], num_classes = label_size, dtype='int8')
        
        model = model_embedding(out_size = label_size)
        if label_size == 2:
            model.compile(loss='binary_crossentropy', optimizer = Adam(lr=0.001), metrics=['acc'])
        else:
            model.compile(loss='categorical_crossentropy', optimizer = Adam(lr=0.001), metrics=['acc'])
        
            
        
        model.fit(x_trn, y_trn, batch_size=32, epochs=10, validation_data=(x_val, y_val), callbacks = cb, verbose=1)
        models.append(model)
        val_pred = model.predict(x_val, batch_size=32, verbose=1)
        oof[val_idx] = val_pred
        test_pred = model.predict(X_test, batch_size=32, verbose=1)
        
        pred += test_pred / kfold.n_splits
  
        print(index+1, 'val acc:', metrics.accuracy_score(y_val, np.argmax(val_pred, axis=1)))
        del x_trn, y_trn, x_val, y_val, val_pred, test_pred
        del model
        gc.collect()
        
    return models, pred, oof

def run_nn(X, y, X_test, label_size, embedding_matrix):
    K.clear_session()
    print('-' * 88)
    x_trn, y_trn = X, to_categorical(y, num_classes = label_size, dtype='int8')
    
    model = model_embedding(label_size, embedding_matrix)
    if label_size == 2:
        model.compile(loss='binary_crossentropy', optimizer = Adam(lr=0.001), metrics=['acc'])
    else:
        model.compile(loss='categorical_crossentropy', optimizer = Adam(lr=0.001), metrics=['acc'])
    
        
    
    model.fit(x_trn, y_trn, batch_size=32, epochs=1, validation_split=0.1, shuffle = True, callbacks = cb, verbose=1)
    pred = model.predict(X_test, batch_size=32, verbose=1)
    
    return pred

In [ ]:
def group_df(df, group_id, feat):
    df[feat] = df[feat].astype(str)
    return df.groupby(group_id)[feat].agg(list)

In [ ]:
# Load data
train_path = "work/train_preliminary/"
train_user = pd.read_csv(train_path + "user.csv")

In [ ]:
import os
if os.path.isfile('work/df_data.csv'):
    df_data = pd.read_csv('work/df_data.csv')
else:
    df_data = genrate_df_data()

data_frame = group_df(df_data, 'user_id', 'creative_id')

In [ ]:
trainIdx = train_user.user_id.values
testIdx = [ix for ix in data_frame.index if ix not in trainIdx]

In [ ]:
trainIdx

In [ ]:
X_train = pad_sequences(np.array(data_frame.iloc[train_user.user_id].sort_index().apply(np.array)), maxlen=maxlen)
X_test = pad_sequences(np.array(data_frame.drop(train_user.user_id, axis = 0).sort_index().apply(np.array)), maxlen=maxlen)
print(X_train.shape, X_test.shape)

(900000, 32) (1000000, 32)


In [ ]:
age_map = {i: i-1 for i in range(1,11)}
age_map_rev = {v:k for k,v in age_map.items()}
gender_map = {1: 1, 2: 0}
gender_map_rev = {v:k for k,v in gender_map.items()}

fold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)

In [ ]:
y_age = train_user[['age','user_id']].set_index("user_id").sort_index()
y_gender = train_user[['gender','user_id']].set_index("user_id").sort_index()

y_age.age = y_age.age.map(age_map)
y_gender.gender = y_gender.gender.map(gender_map)

In [ ]:
age_models, age_pred, age_oof = run_oof_nn(X_train, y_age, X_test, fold, label_size = 10)

NameError: name 'run_oof_nn' is not defined

In [ ]:
train_path = "work/train_preliminary/"
test_path = "work/test/"

In [ ]:
train_ad = pd.read_csv(train_path + "ad.csv")
train_click_log = pd.read_csv(train_path + "click_log.csv")
train_user = pd.read_csv(train_path + "user.csv")

In [ ]:
test_ad = pd.read_csv(test_path + "ad.csv")
test_click_log = pd.read_csv(test_path + "click_log.csv")

In [ ]:
Feats = pd.read_csv('features/300_w2v_advertiser_id_feats.csv', index_col = 'user_id')

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
Feats

,Unnamed: 0,embedding_advertiser_id_0,embedding_advertiser_id_1,embedding_advertiser_id_2,embedding_advertiser_id_3,embedding_advertiser_id_4,embedding_advertiser_id_5,embedding_advertiser_id_6,embedding_advertiser_id_7,embedding_advertiser_id_8,...,embedding_advertiser_id_290,embedding_advertiser_id_291,embedding_advertiser_id_292,embedding_advertiser_id_293,embedding_advertiser_id_294,embedding_advertiser_id_295,embedding_advertiser_id_296,embedding_advertiser_id_297,embedding_advertiser_id_298,embedding_advertiser_id_299
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0.101602,-0.079968,-0.092134,-0.054349,0.130472,0.132915,0.010918,-0.024641,0.124037,...,0.058038,-0.001717,0.064998,0.018734,0.132311,0.028684,-0.071284,-0.117009,-0.001652,0.195771
2,1,0.021537,-0.167583,-0.048966,-0.097469,0.135841,0.097632,-0.006470,-0.068229,0.177142,...,0.088446,0.036978,0.034290,-0.047635,0.108675,-0.060901,-0.134916,-0.160036,0.028715,0.217165
3,2,0.042490,-0.143809,-0.096629,-0.000706,0.147389,0.082866,-0.032475,-0.047307,0.192894,...,0.020306,0.111382,0.076355,0.010171,0.022357,-0.149842,-0.008776,-0.213509,0.130689,0.198206
4,3,0.054780,-0.171053,-0.075172,-0.026324,0.092897,0.117008,0.020993,0.004142,0.261204,...,0.053216,0.021665,0.020760,-0.012131,0.083061,-0.060756,-0.092258,-0.160074,0.094474,0.304928
5,4,0.031284,-0.197200,-0.201585,-0.055078,0.117486,0.085931,-0.045597,0.021419,0.151421,...,0.084966,0.064698,-0.009051,-0.032145,0.084221,0.078474,0.022829,-0.094970,-0.061097,0.273893
6,5,0.012289,0.029437,-0.096532,-0.083201,0.101368,-0.005779,-0.030008,0.046715,0.284621,...,0.011363,0.008301,0.149666,-0.099009,0.089201,-0.000186,-0.043077,-0.103919,0.100386,0.223032
7,6,0.065622,-0.157877,-0.128865,-0.050445,0.216643,0.065458,0.000473,-0.009831,0.096775,...,0.072286,0.107247,0.106642,-0.072508,0.142935,-0.026571,-0.080213,-0.184293,0.029159,0.233668
8,7,0.042247,-0.093159,-0.060192,-0.087350,0.177095,0.064074,-0.011089,0.012863,0.212384,...,0.033661,-0.053804,0.066015,-0.041498,0.062905,-0.038275,-0.075872,-0.215895,0.012626,0.214004
9,8,0.050872,-0.287499,-0.008635,-0.155768,0.164503,-0.037841,0.077350,0.003839,0.205544,...,0.052785,-0.078424,0.012340,-0.078897,0.168296,-0.013203,-0.017840,-0.204774,0.047632,0.149671


In [ ]:
Feats.set_index('user_id').iloc[train_user.user_id].sort_index()

,creative_id_max,creative_id_min,creative_id_mean,creative_id_sum,creative_id_std,creative_id_nunique,creative_id_count,ad_id_max,ad_id_min,ad_id_mean,...,time_std,time_nunique,time_count,click_times_max,click_times_min,click_times_mean,click_times_sum,click_times_std,click_times_nunique,click_times_count
user_id,,,,,,,,,,,,,,,,,,,,,
1,3592928.0,71691.0,1.498318e+06,19478128.0,1.213850e+06,12,13,3093461.0,66210.0,1.296254e+06,...,19.585775,10,13,2.0,1.0,1.076923,14.0,0.277350,2,13
2,4194247.0,13069.0,1.626222e+06,73180007.0,1.211914e+06,42,45,3598073.0,14495.0,1.406817e+06,...,21.653562,28,45,2.0,1.0,1.022222,46.0,0.149071,2,45
3,3696597.0,66009.0,1.881833e+06,56454986.0,1.233167e+06,30,30,3180793.0,60437.0,1.600597e+06,...,26.386953,23,30,1.0,1.0,1.000000,30.0,0.000000,1,30
4,4039445.0,31070.0,1.894423e+06,54938256.0,1.124894e+06,29,29,3467430.0,30773.0,1.635741e+06,...,17.421039,15,29,1.0,1.0,1.000000,29.0,0.000000,1,29
5,4189888.0,24333.0,1.872902e+06,61805775.0,1.168669e+06,33,33,3594306.0,24966.0,1.618947e+06,...,23.186783,26,33,2.0,1.0,1.030303,34.0,0.174078,2,33
6,4266044.0,11212.0,1.868821e+06,31769949.0,1.134700e+06,16,17,3659646.0,12640.0,1.616687e+06,...,17.666852,14,17,1.0,1.0,1.000000,17.0,0.000000,1,17
7,4096991.0,7054.0,1.461038e+06,14610384.0,1.376587e+06,10,10,3516020.0,8466.0,1.263590e+06,...,27.773088,10,10,1.0,1.0,1.000000,10.0,0.000000,1,10
8,4289670.0,5275.0,2.194046e+06,107508271.0,1.326272e+06,48,49,3679223.0,6499.0,1.892245e+06,...,24.627566,34,49,2.0,1.0,1.061224,52.0,0.242226,2,49
9,3042304.0,88325.0,1.714886e+06,34297712.0,9.400354e+05,17,20,2616285.0,80848.0,1.484352e+06,...,19.413709,17,20,2.0,1.0,1.050000,21.0,0.223607,2,20


In [ ]:
Feats.set_index('user_id').sort_index()

,creative_id_max,creative_id_min,creative_id_mean,creative_id_sum,creative_id_std,creative_id_nunique,creative_id_count,ad_id_max,ad_id_min,ad_id_mean,...,time_std,time_nunique,time_count,click_times_max,click_times_min,click_times_mean,click_times_sum,click_times_std,click_times_nunique,click_times_count
user_id,,,,,,,,,,,,,,,,,,,,,
1,3592928.0,71691.0,1.498318e+06,19478128.0,1.213850e+06,12,13,3093461.0,66210.0,1.296254e+06,...,19.585775,10,13,2.0,1.0,1.076923,14.0,0.277350,2,13
2,4194247.0,13069.0,1.626222e+06,73180007.0,1.211914e+06,42,45,3598073.0,14495.0,1.406817e+06,...,21.653562,28,45,2.0,1.0,1.022222,46.0,0.149071,2,45
3,3696597.0,66009.0,1.881833e+06,56454986.0,1.233167e+06,30,30,3180793.0,60437.0,1.600597e+06,...,26.386953,23,30,1.0,1.0,1.000000,30.0,0.000000,1,30
4,4039445.0,31070.0,1.894423e+06,54938256.0,1.124894e+06,29,29,3467430.0,30773.0,1.635741e+06,...,17.421039,15,29,1.0,1.0,1.000000,29.0,0.000000,1,29
5,4189888.0,24333.0,1.872902e+06,61805775.0,1.168669e+06,33,33,3594306.0,24966.0,1.618947e+06,...,23.186783,26,33,2.0,1.0,1.030303,34.0,0.174078,2,33
6,4266044.0,11212.0,1.868821e+06,31769949.0,1.134700e+06,16,17,3659646.0,12640.0,1.616687e+06,...,17.666852,14,17,1.0,1.0,1.000000,17.0,0.000000,1,17
7,4096991.0,7054.0,1.461038e+06,14610384.0,1.376587e+06,10,10,3516020.0,8466.0,1.263590e+06,...,27.773088,10,10,1.0,1.0,1.000000,10.0,0.000000,1,10
8,4289670.0,5275.0,2.194046e+06,107508271.0,1.326272e+06,48,49,3679223.0,6499.0,1.892245e+06,...,24.627566,34,49,2.0,1.0,1.061224,52.0,0.242226,2,49
9,3042304.0,88325.0,1.714886e+06,34297712.0,9.400354e+05,17,20,2616285.0,80848.0,1.484352e+06,...,19.413709,17,20,2.0,1.0,1.050000,21.0,0.223607,2,20


In [ ]:
Feats.set_index('user_id').drop(train_user.user_id, axis = 0).sort_index()

,creative_id_max,creative_id_min,creative_id_mean,creative_id_sum,creative_id_std,creative_id_nunique,creative_id_count,ad_id_max,ad_id_min,ad_id_mean,...,time_std,time_nunique,time_count,click_times_max,click_times_min,click_times_mean,click_times_sum,click_times_std,click_times_nunique,click_times_count
user_id,,,,,,,,,,,,,,,,,,,,,
3000001,3899689.0,103064.0,1.594291e+06,17537198.0,1.120795e+06,11,11,3350034.0,93662.0,1.380382e+06,...,24.080169,7,11,1.0,1.0,1.000000,11.0,0.000000,1,11
3000002,4131664.0,12838.0,1.458852e+06,88989989.0,1.423655e+06,49,61,3544856.0,14271.0,1.261534e+06,...,24.755107,37,61,2.0,1.0,1.065574,65.0,0.249590,2,61
3000003,4065209.0,20503.0,1.608424e+06,57903264.0,1.363916e+06,35,36,3489224.0,21487.0,1.390366e+06,...,29.832495,23,36,1.0,1.0,1.000000,36.0,0.000000,1,36
3000004,3220837.0,9870.0,1.144744e+06,22894884.0,1.150346e+06,17,20,2770924.0,11324.0,9.930211e+05,...,21.509545,15,20,1.0,1.0,1.000000,20.0,0.000000,1,20
3000005,3863936.0,17535.0,1.512585e+06,42352371.0,1.315763e+06,23,28,3320090.0,18818.0,1.306084e+06,...,30.234074,24,28,2.0,1.0,1.035714,29.0,0.188982,2,28
3000006,4120127.0,126368.0,2.182746e+06,34923930.0,1.441273e+06,16,16,3535336.0,113773.0,1.882972e+06,...,29.284737,12,16,4.0,1.0,1.312500,21.0,0.793200,3,16
3000007,3992926.0,165900.0,1.752662e+06,17526617.0,1.431892e+06,10,10,3428744.0,148702.0,1.519014e+06,...,28.744082,8,10,1.0,1.0,1.000000,10.0,0.000000,1,10
3000008,4273932.0,33944.0,1.516556e+06,42463579.0,1.153818e+06,26,28,3665878.0,33062.0,1.312563e+06,...,19.831399,24,28,3.0,1.0,1.107143,31.0,0.416270,3,28
3000009,3856981.0,40691.0,1.863434e+06,44722428.0,1.366552e+06,23,24,3314321.0,38902.0,1.608416e+06,...,25.671701,19,24,2.0,1.0,1.041667,25.0,0.204124,2,24


In [ ]:
train_user[['age','user_id']].set_index("user_id").sort_index()

,age
user_id,
1,4
2,10
3,7
4,5
5,4
6,6
7,6
8,5
9,5


In [ ]:
# df_w2v = pd.read_csv('./features/300_w2v_creative_id_feats.csv', index_col = 'user_id')

In [ ]:
# del df_w2v['Unnamed: 0']

In [ ]:
# df_w2v

In [ ]:
# 训练数据数据的 ad click_log 和 用户人数
train_ad.shape, train_click_log.shape, train_user.shape, len(train_click_log.user_id.unique())

((2481135, 6), (30082771, 4), (900000, 3), 900000)

In [ ]:
# 测试数据的 ad click_log 和 用户人数
test_ad.shape, test_click_log.shape, len(test_click_log.user_id.unique())

((2618159, 6), (33585512, 4), 1000000)

In [ ]:
train_ad.head(2)

,creative_id,ad_id,product_id,product_category,advertiser_id,industry
0,1,1,\N,5,381,78
1,4,4,\N,5,108,202


In [ ]:
train_click_log.tail(2)

,time,user_id,creative_id,click_times
30082769,86,30920,2713031,1
30082770,25,30920,629802,1


In [ ]:
train_user.head(2)

,user_id,age,gender
0,1,4,1
1,2,10,1


In [ ]:
# # train_user 和 train_click_log 的 user_id 是一致的
# set(train_user.user_id) == set(train_click_log.user_id)

In [ ]:
# # train_ad 和 train_click_log 的 creative_id 是一致的
# len(set(train_ad.creative_id)), len(set(train_click_log.creative_id))
# set(train_ad.creative_id) == set(train_click_log.creative_id)

In [ ]:
test_click_log.head(2)

,time,user_id,creative_id,click_times
0,20,3131989,645764,1
1,20,3131989,1027422,1


In [ ]:
df_train = train_ad.merge(train_click_log, on = "creative_id", how = "left")

In [ ]:
df_test = test_ad.merge(test_click_log, on = "creative_id", how = "left")

In [ ]:
df_train.head(2)

,creative_id,ad_id,product_id,product_category,advertiser_id,industry,time,user_id,click_times
0,1,1,\N,5,381,78,81,398695,1
1,1,1,\N,5,381,78,82,404020,1


In [ ]:
df_test.head(2)

,creative_id,ad_id,product_id,product_category,advertiser_id,industry,time,user_id,click_times
0,1,1,\N,5,381,78,81,3153317,1
1,1,1,\N,5,381,78,81,3284714,1


In [ ]:
df_data = pd.concat([df_train, df_test], ignore_index = True)

In [ ]:
df_data = df_data.replace('\\N', -1)

In [ ]:
df_data.to_csv('work/df_data.csv', index = False)

In [ ]:
data_frame = df_data.groupby('user_id')['creative_id'].agg(list).reset_index()

In [ ]:
data_frame

### EDA

In [ ]:
# # 缺失值是 '\\N', 先替换成nan.
# df_data = df_data.replace('\\N', np.nan)

In [ ]:
# 缺失值
# (df_data.isnull().sum()).plot.bar(title='Data null')

In [ ]:
# plt.rcParams['font.sans-serif']=['SimHei']
# train_user['age'].value_counts().plot.pie(autopct='%1.1f%%',title = '年龄分布比例')

In [ ]:
# # 性别比例大约是2:1
# train_user['gender'].value_counts().plot.bar(title = '性别分布比例')

In [ ]:
# df_data[["creative_id", "ad_id", "user_id"]].boxplot()

In [ ]:
# df_data[["advertiser_id", "product_category", "time", "click_times"]].boxplot()

### 提取特征

In [ ]:
# # 缺失值用-1补充
# df_data = df_data.fillna(-1)

In [ ]:
# for col in df_data.columns:
#     print(col, ":", type(df_data[col][0]))

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from gensim.models import Word2Vec
# for col in ["creative_id", "ad_id", "product_category", "advertiser_id", "time", "click_times"]:
#     le = LabelEncoder()
#     df_data[col] = le.fit_transform(df_data[col])

In [ ]:
Feats = df_data.drop_duplicates(subset = "user_id")[["user_id"]]

In [ ]:
# del df_data
# gc.collect()

In [ ]:
#构造统计特征
def group_feature(df, key, target, aggs):   
    agg_dict = {}
    for ag in aggs:
        agg_dict[f'{target}_{ag}'] = ag
    print(agg_dict)
    df[target] = df[target].astype(float)
    t = df.groupby(key)[target].agg(agg_dict).reset_index()
    return t

In [ ]:
# for col in ["creative_id", "ad_id", "product_category", "advertiser_id", "time", "click_times"]:
#     t = group_feature(df_data, "user_id", col, ['max','min','mean','sum','std', 'nunique', 'count'])
#     Feats = Feats.merge(t, on = "user_id", how = "left")

In [ ]:
def get_count_tfidf(df, group_id, group_target, num):
    df[group_target] = df[group_target].astype(str)
    tmp = df.groupby(group_id)[group_target].agg(list).reset_index()
    tmp[group_target] = tmp[group_target].apply(lambda x: ' '.join(x))

    tfidf_enc_tmp = TfidfVectorizer(max_features = 100000, min_df = 3)
    tfidf_vec_tmp = tfidf_enc_tmp.fit_transform(tmp[group_target])
    svd_tag_tmp = TruncatedSVD(n_components=num, n_iter=5, random_state=52)
    tag_svd_tmp = svd_tag_tmp.fit_transform(tfidf_vec_tmp)
    tag_svd_tmp = pd.DataFrame(tag_svd_tmp)
    tag_svd_tmp.columns = ['{}_tfidf_{}'.format(group_target, i)
                           for i in range(num)]

    countvec = CountVectorizer(max_features = 100000, min_df = 3)
    count_vec_tmp = countvec.fit_transform(tmp[group_target])
    svd_tmp = TruncatedSVD(n_components=num, n_iter=5, random_state=52)
    svd_tmp = svd_tmp.fit_transform(count_vec_tmp)
    svd_tmp = pd.DataFrame(svd_tmp)
    svd_tmp.columns = ['{}_countvec_{}'.format(group_target, i)
                       for i in range(num)]

    return pd.concat([tmp[[group_id]], tag_svd_tmp, svd_tmp], axis=1)

In [ ]:
# # for col in ["creative_id", "ad_id", "advertiser_id"]:
# for col in ["advertiser_id"]:
#     t = get_count_tfidf(df_data, "user_id", col, 30)
#     Feats = Feats.merge(t, on = "user_id", how = "left")

In [ ]:
def hashfxn(astring):
    return ord(astring[0])
def w2v_feat(df, group_id, feat, length):
    df[feat] = df[feat].astype(str)
    data_frame = df.groupby(group_id)[feat].agg(list).reset_index()
    model = Word2Vec(data_frame[feat].values,
                     size=length,
                     window= 30,
                     min_count = 5,
                     workers = 8,
                     sg = 1,
                     iter = 5,
                     seed = 1,
                     hashfxn = hashfxn)
    data_frame[feat] = data_frame[feat].apply(
        lambda x: np.array([[model.wv[c] if c in model.wv else [0]*length] for c in x])
    )
    
    for m in range(length):
        data_frame['w2v_{}_mean'.format(m)] = data_frame[feat].apply(
            lambda x: x[:, m].mean()
        )
    del data_frame[feat]
    gc.collect()
    return data_frame

In [ ]:
# for col in ["creative_id", "ad_id", "advertiser_id"]:
for col in ["creative_id"]:
    t = w2v_feat(df_data, "user_id", col, 100)
    Feats = Feats.merge(t, on = "user_id", how = "left")

KeyboardInterrupt: 

In [ ]:
feat_files = ['creative_id_w2v_feats.csv', 'ad_id_w2v_feats.csv', 'advertiser_id_w2v_feats.csv',
    'ad_id_tfidf_feats.csv', 'advertiser_id_tfidf_feats.csv', 'creative_id_tfidf_feats.csv', 'group_feats.csv']

In [ ]:
# for df_file in feat_files:
#     t = pd.read_csv(df_file)
#     Feats = Feats.merge(t, on = "user_id", how = "left")
#     del t
#     gc.collect()

In [ ]:
Feats = Feats.set_index('user_id')

In [ ]:
Feats.to_csv("./300_creative_id_w2c_feats.csv")

In [ ]:
X_train = Feats.iloc[train_user.user_id].sort_index()

In [ ]:
X_test = Feats.drop(train_user.user_id, axis = 0).sort_index()

In [ ]:
y_age = train_user[['age','user_id']].set_index("user_id")
y_gender = train_user[['gender','user_id']].set_index("user_id")

In [ ]:
age_map = {i: i-1 for i in range(1,11)}
age_map_rev = {v:k for k,v in age_map.items()}
gender_map = {1: 1, 2: 0}
gender_map_rev = {v:k for k,v in gender_map.items()}

In [ ]:
y_age.age = y_age.age.map(age_map)
y_gender.gender = y_gender.gender.map(gender_map)

In [ ]:
X_train.shape, X_test.shape, y_age.shape, y_gender.shape

### Model

In [ ]:
# import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics

In [ ]:
# lgb_params_age = {
#     'learning_rate' : 0.1,
#     # 'min_child_samples': 5,
#     'max_depth': 7,
#     'lambda_l1': 2,
#     # 'feature_fraction': .75,
#     # 'bagging_fraction': .85,
#     # 'seed': 99,
#     'n_estimators': 3000,
#     'boosting_type': 'gbdt',
#     'objective': 'multiclass',
#     'num_class': 10,
#     'early_stopping_rounds': 100,
# }
# fold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
# models_age = []
# pred_age = np.zeros((X_test.shape[0], 10))
# oof_age = np.zeros((X_train.shape[0], 10))

In [ ]:
# for index, (train_idx, val_idx) in enumerate(fold.split(X_train, y_age)):

#     train_set = lgb.Dataset(X_train.iloc[train_idx], y_age.iloc[train_idx])
#     val_set = lgb.Dataset(X_train.iloc[val_idx], y_age.iloc[val_idx])

#     model = lgb.train(lgb_params_age, train_set, valid_sets=[train_set, val_set],verbose_eval=100)
#     models_age.append(model)

#     val_pred = model.predict(X_train.iloc[val_idx])
#     oof_age[val_idx] = val_pred
#     val_y = y_age.age.values[val_idx]
#     val_pred = np.argmax(val_pred, axis = 1)

#     print(index+1, 'val acc:', metrics.accuracy_score(val_y, val_pred))
#     test_pred = model.predict(X_test)
#     pred_age += test_pred/fold.n_splits
#     del train_set, val_set, val_pred, val_y, test_pred
#     gc.collect()

# # 372feats : 42 stat, 150 w2v, 180 tf_idf
# # 线下 -
# # 线上 0.3215

In [ ]:
# lgb_params_gender = {
#     'n_estimators': 3000,
#     'boosting_type': 'gbdt',
#     'objective': 'binary',
#     # 'metric': 'None',
#     # 'num_leaves': 63,
#     # 'subsample': 0.8,
#     # 'colsample_bytree': 0.8,
#     'learning_rate': 0.05,
#     'lambda_l2':2,
#     'nthread': -1,
#     # 'silent': True,
#     'early_stopping_rounds': 100,
# }
# models_gender = []
# oof_gender = np.zeros(X_train.shape[0])
# pred_gender = np.zeros(X_test.shape[0])

In [ ]:
# for index, (train_idx, val_idx) in enumerate(fold.split(X_train, y_gender)):

#     train_set = lgb.Dataset(X_train.iloc[train_idx], y_gender.iloc[train_idx])
#     val_set = lgb.Dataset(X_train.iloc[val_idx], y_gender.iloc[val_idx])

#     model = lgb.train(lgb_params_gender, train_set, valid_sets=[train_set, val_set], verbose_eval=100)
#     models_gender.append(model)
#     val_pred = model.predict(X_train.iloc[val_idx])
#     oof_gender[val_idx] = val_pred
#     val_y = y_gender.values.reshape((-1))[val_idx]
#     val_pred = np.round(val_pred)
    
#     print(index+1, 'val acc:', metrics.accuracy_score(val_y, val_pred))
#     test_pred = model.predict(X_test)
#     pred_gender += test_pred/fold.n_splits
#     del train_set, val_set, val_pred, val_y, test_pred
#     gc.collect()
# # 372feats : 42 stat, 150 w2v, 180 tf_idf
# # 线下 0.717
# # 线上 0.876774

In [ ]:
import sys
sys.path.append('/home/aistudio/external-libraries/lib/python3.7/site-packages')
import catboost as cbt

In [ ]:
cbt_params_age={
    "iterations":10000,
      "use_best_model":True,
    "random_seed":2020,
     "logging_level":'Verbose', 
    "early_stopping_rounds":200, 
    "loss_function":'MultiClass',
    'eval_metric':'MultiClass',
    'task_type':'GPU',
     'devices':'0'
}

fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
models_age = []
pred_age = np.zeros((X_test.shape[0], 10))
oof_age = np.zeros((X_train.shape[0], 10))

In [ ]:
%%time
for index, (trn_idx, val_idx) in enumerate(fold.split(X=X_train, y=y_age)):
    print('-' * 88)
    x_trn, y_trn = X_train.iloc[trn_idx], y_age.iloc[trn_idx]
    x_val, y_val = X_train.iloc[val_idx], y_age.iloc[val_idx]
    
    trn_pool = cbt.Pool(x_trn, y_trn)
    val_pool = cbt.Pool(x_val, y_val)
    model = cbt.CatBoostClassifier(**cbt_params_age)
    model.fit(trn_pool, eval_set=val_pool, verbose=500)
    models_age.append(model)
    val_pred = model.predict_proba(x_val)
    oof_age[val_idx] = val_pred

    val_pred = np.argmax(val_pred, axis=1)
    pred_age += (model.predict_proba(X_test) / fold.n_splits)

    print(index+1, 'val acc:', metrics.accuracy_score(y_val, val_pred))
    del x_trn, y_trn, x_val, y_val
    del trn_pool, val_pool
    gc.collect()
# # 372feats : 42 stat, 150 w2v, 180 tf_idf
# # 线下 0.252
# # 线上 0.876774

In [ ]:
cbt_params_gender={
    "iterations":10000,
      "use_best_model":True,
    "random_seed":2020,
     "logging_level":'Verbose', 
    "early_stopping_rounds":200, 
    "loss_function":'Logloss',
    'eval_metric':'AUC',
    'task_type':'GPU',
     'devices':'0'
}

fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
models_gender = []
pred_gender = np.zeros((X_test.shape[0], 2))
oof_gender = np.zeros((X_train.shape[0], 2))

In [ ]:
%%time
for index, (trn_idx, val_idx) in enumerate(fold.split(X=X_train, y=y_gender)):
    print('-' * 88)
    x_trn, y_trn = X_train.iloc[trn_idx], y_gender.iloc[trn_idx]
    x_val, y_val = X_train.iloc[val_idx], y_gender.iloc[val_idx]
    
    trn_pool = cbt.Pool(x_trn, y_trn)
    val_pool = cbt.Pool(x_val, y_val)
    model = cbt.CatBoostClassifier(**cbt_params_gender)
    model.fit(trn_pool, eval_set=val_pool, verbose=500)
    models_gender.append(model)
    val_pred = model.predict_proba(x_val)
    oof_gender[val_idx] = val_pred

    val_pred = np.argmax(val_pred, axis=1)
    pred_gender += (model.predict_proba(X_test) / fold.n_splits)

    print(index+1, 'val acc:', metrics.accuracy_score(y_val, val_pred))
    del x_trn, y_trn, x_val, y_val
    del trn_pool, val_pool
    gc.collect()
# # 372feats : 42 stat, 150 w2v, 180 tf_idf
# # 线下 0.718
# # 线上 0.876774

In [ ]:
np.unique(np.argmax(pred_age, axis=1))

In [ ]:
np.unique(np.argmax(pred_gender, axis=1))

### Feature importance

In [ ]:
ret = []
for index, model in enumerate(models_age):
    df = pd.DataFrame()
    '''
    # lgb
    df['name'] = model.feature_name()
    df['score'] = model.feature_importance()
    df['fold'] = index
    '''
    # catboost
    df['name'] = model.feature_names_
    df['score'] = model.feature_importances_
    df['fold'] = index
    ret.append(df)
    
df = pd.concat(ret)

df = df.groupby('name', as_index=False)['score'].mean()
df.sort_values(['score'], ascending=False)

In [ ]:
ret = []
for index, model in enumerate(models_gender):
    df = pd.DataFrame()
    '''
    # lgb
    df['name'] = model.feature_name()
    df['score'] = model.feature_importance()
    df['fold'] = index
    '''
    # catboost
    df['name'] = model.feature_names_
    df['score'] = model.feature_importances_
    df['fold'] = index
    ret.append(df)
    
df = pd.concat(ret)

df = df.groupby('name', as_index=False)['score'].mean()
df.sort_values(['score'], ascending=False)

### Submission

In [ ]:
sub = pd.DataFrame()

In [ ]:
sub['user_id'] = X_test.index
sub['predicted_age'] = np.argmax(pred_age, axis = 1)
# sub['predicted_age'] = -1
sub['predicted_gender'] = np.argmax(pred_gender, axis = 1).astype(int)

In [ ]:
sub["predicted_gender"] = sub["predicted_gender"].map(gender_map_rev)
sub["predicted_age"] = sub["predicted_age"].map(age_map_rev)

In [ ]:
sub["predicted_gender"].value_counts()

In [ ]:
sub["predicted_age"].value_counts()

In [ ]:
sub.to_csv("./submission.csv", header = True, index = False, encoding='utf-8')